# Touchdesigner and Mediapipe Integration
This documentation is a walkthrough to setting up mediapipe and integrate it with touchdesigner.

## Requirements
1. __[Touchdesigner 64-Bit Build 2021.16410](https://derivative.ca/download)__
2. __[Python 3.7.2](https://www.python.org/downloads/release/python-372/)__


## Installing Mediapipe and Touchdesign
1. Open command prompt and enter following command
```bash
pip install mediapipe
```
2. Open Touchdesigner and navigate to File>Preferences and enter path to python site-packages on your system usually. For windows the default is in following directory (change username to yours).
```bash
C:\Users\<username>\AppData\Local\Programs\Python\Python37\Lib\site-packages
```

3. Uncheck **Search External Python Path Last**
![001.touchdesigner-preferences.png](img/001.touchdesigner-preferences.png)

4. Save the preferences and restart Touchdesigner.

5. Open Dialogs>Texport and DATs (shortcut alt + t) and enter following command to verify mediapipe installation
```python
import mediapipe as mp
print(dir(mp.solutions))
```
```bash
output:
['__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__', 'download_utils', 'drawing_styles', 'drawing_utils', 'face_detection', 'face_mesh', 'face_mesh_connections', 'hands', 'hands_connections', 'holistic', 'mediapipe', 'objectron', 'pose', 'pose_connections', 'selfie_segmentation']
```

Installation of mediapipe is ready and can be used

## Sample Program
### Face Detection Using Mediapipe in Touchdesigner

In [26]:
import cv2
import mediapipe as mp

#### Get Realtime Webcam Feed

In [27]:
cap = cv2.VideoCapture(0)
while cap.isOpened():
    ret, frame = cap.read()
    cv2.imshow('Raw Webcam Feed', frame)
    
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

In [28]:
cap.release()
cv2.destroyAllWindows()

#### Using Face Detection in Jupyter Notebook

In [29]:
mp_face_detection = mp.solutions.face_detection
mp_drawing = mp.solutions.drawing_utils
cap = cv2.VideoCapture(0)

with mp_face_detection.FaceDetection(
    model_selection=0, min_detection_confidence=0.5) as face_detection:
  while cap.isOpened():
    success, image = cap.read()
    if not success:
      print("Ignoring empty camera frame.")
      # If loading a video, use 'break' instead of 'continue'.
      continue

    # To improve performance, optionally mark the image as not writeable to
    # pass by reference.
    image.flags.writeable = False
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = face_detection.process(image)

    # Draw the face detection annotations on the image.
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    
    if results.detections:
      for detection in results.detections:
        mp_drawing.draw_detection(image, detection)
    
    # Flip the image horizontally for a selfie-view display.
    cv2.imshow('MediaPipe Face Detection', cv2.flip(image, 1))
    
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

#### Using Face Detection in Touchdesigner (CHOP)

Open face_detection.toe inside toe folder for sample. Using CHOP Script is useful if we want to extract position data from mediapipe such as position of landmark. 

Code snippet from CHOP Script
```python
# me - this DAT
# scriptOp - the OP which is cooking

import numpy as np
import cv2
import sys
import mediapipe as mp

mp_face = mp.solutions.face_detection
face_detection = mp_face.FaceDetection(
    model_selection=0,
    min_detection_confidence=0.7
)

# press 'Setup Parameters' in the OP to call this function to re-create the parameters.
def onSetupParameters(scriptOp):
    page = scriptOp.appendCustomPage('Custom')
    topPar = page.appendTOP('Face', label='Image with face')
    return

# called whenever custom pulse parameter is pushed
def onPulse(par):
	return

def onCook(scriptOp):
    scriptOp.clear()
    topRef = scriptOp.par.Face.eval()
    
    num_faces = 0
    max_area = sys.float_info.min
    width = 0
    height = 0
    xmin = 0.5
    ymin = 0.5
    lx = sys.float_info.max
    ly = sys.float_info.max
    rx = sys.float_info.max
    ry = sys.float_info.max
    
    if topRef:
        img = topRef.numpyArray(delayed=True)
        frame = cv2.cvtColor(img, cv2.COLOR_RGBA2RGB)
        frame *= 255
        frame = frame.astype('uint8')
        results = face_detection.process(frame)
        
        if results.detections:
            num_faces = len(results.detections)
            
            for face in results.detections:
            	area = face.location_data.relative_bounding_box.width * face.location_data.relative_bounding_box.height
            	if area > max_area:
            		width = face.location_data.relative_bounding_box.width
            		height = face.location_data.relative_bounding_box.height
            		xmin = face.location_data.relative_bounding_box.xmin + width/2.0
            		ymin = 1 - (face.location_data.relative_bounding_box.ymin + height/2.0)
            		lx = face.location_data.relative_keypoints[0].x
            		ly = 1 - face.location_data.relative_keypoints[0].y
            		rx = face.location_data.relative_keypoints[1].x
            		ry = 1 - face.location_data.relative_keypoints[1].y
            		max_area = area
        
    tf = scriptOp.appendChan('face')
    tw = scriptOp.appendChan('width')
    th = scriptOp.appendChan('height')
    tx = scriptOp.appendChan('tx')
    ty = scriptOp.appendChan('ty')
    leftx = scriptOp.appendChan('left_eye_x')
    lefty = scriptOp.appendChan('left_eye_y')
    rightx = scriptOp.appendChan('right_eye_x')
    righty = scriptOp.appendChan('right_eye_y')
    
    tf.vals = [num_faces]
    tw.vals = [width]
    th.vals = [height]
    tx.vals = [xmin]
    ty.vals = [ymin]
    
    leftx.vals = [lx]
    lefty.vals = [ly]
    rightx.vals = [rx]
    righty.vals = [ry]
    
    scriptOp.rate = me.time.rate

    return
```

#### Using Face Mesh in Touchdesigner (TOP)

```python
# me - this DAT
# scriptOp - the OP which is cooking

import numpy
import cv2
import mediapipe as mp

mp_drawing = mp.solutions.drawing_utils
mp_face_mesh = mp.solutions.face_mesh

point_spec = mp_drawing.DrawingSpec(
	color=(0, 100, 255),
    thickness=1,
    circle_radius=1
)
line_spec = mp_drawing.DrawingSpec(
	color=(255, 200, 0),
    thickness=2,
    circle_radius=1
)
face_mesh = mp_face_mesh.FaceMesh(
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
)

# press 'Setup Parameters' in the OP to call this function to re-create the parameters.
def onSetupParameters(scriptOp):
	return

# called whenever custom pulse parameter is pushed
def onPulse(par):
	return


def onCook(scriptOp):
	input = scriptOp.inputs[0].numpyArray(delayed=True)
	if input is not None:
		frame = input * 255
		frame = frame.astype('uint8')
		frame = cv2.cvtColor(frame, cv2.COLOR_RGBA2RGB)
		
        results = face_mesh.process(frame)
		
        if results.multi_face_landmarks:
			for face_landmarks in results.multi_face_landmarks:
				mp_drawing.draw_landmarks(
					image=frame,
					landmark_list=face_landmarks,
					connections=mp_face_mesh.FACEMESH_TESSELATION,
					landmark_drawing_spec=point_spec,
					connection_drawing_spec=line_spec)

		frame = cv2.cvtColor(frame, cv2.COLOR_RGB2RGBA)
		scriptOp.copyNumpyArray(frame)
	return
```

#### Using Hands in Touchdesigner

```python
# me - this DAT
# scriptOp - the OP which is cooking

import numpy
import cv2
import mediapipe as mp

# setup for mediapipe
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(	model_complexity=0,
                        min_detection_confidence=0.5,
                        min_tracking_confidence=0.5)

# press 'Setup Parameters' in the OP to call this function to re-create the parameters.
def onSetupParameters(scriptOp):
    return

# called whenever custom pulse parameter is pushed
def onPulse(par):
    return

def onCook(scriptOp):
    input = scriptOp.inputs[0].numpyArray(delayed=True)

    if input is not None:
        image = input * 255
        image = image.astype('uint8')
        image = cv2.cvtColor(image, cv2.COLOR_RGBA2RGB)
    
        results = hands.process(image)
        
        # Draw the hand annotations on the image.	
        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                mp_drawing.draw_landmarks(
                    image,
                    hand_landmarks,
                    mp_hands.HAND_CONNECTIONS,
                    mp_drawing_styles.get_default_hand_landmarks_style(),
                    mp_drawing_styles.get_default_hand_connections_style())
        
        scriptOp.copyNumpyArray(image)
    
    return
```

#### Using Hands in Jupyter Notebook

In [30]:
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_hands = mp.solutions.hands

# For webcam input:
cap = cv2.VideoCapture(0)
with mp_hands.Hands(
    model_complexity=0,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5) as hands:
  while cap.isOpened():
    success, image = cap.read()
    if not success:
      print("Ignoring empty camera frame.")
      # If loading a video, use 'break' instead of 'continue'.
      continue

    # To improve performance, optionally mark the image as not writeable to
    # pass by reference.
    image.flags.writeable = False
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = hands.process(image)

    # Draw the hand annotations on the image.
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    if results.multi_hand_landmarks:
      for hand_landmarks in results.multi_hand_landmarks:
        mp_drawing.draw_landmarks(
            image,
            hand_landmarks,
            mp_hands.HAND_CONNECTIONS,
            mp_drawing_styles.get_default_hand_landmarks_style(),
            mp_drawing_styles.get_default_hand_connections_style())
    
    # Flip the image horizontally for a selfie-view display.
    cv2.imshow('MediaPipe Hands', cv2.flip(image, 1))
    
    if cv2.waitKey(10) & 0xFF == ord('q'):
      break
    
cap.release()
cv2.destroyAllWindows()

#### Notes on Using Mediapipe in Touchdesigner

1. It is relatively easy to install and setup mediapipe for touchdesigner
2. Integration only require you to specify path to mediapipe installation (site-packages folder)
3. Touchdesigner has built-in python version 3.7.2, to minimize compatibility issue I think it is recommended to install python 3.7.2 in your system and install mediapipe globally in your system.
4. I noticed performance drop when executing mediapipe code from touchdesigner. I used Intel Core i7 8550U with NVDIA MX150 and 8 GB of RAM. I saw 7-10 fps drop when executing same code in touchdesigner compared to executing it from command prompt. It is likely to another process running in touchdesigner, I expect it doesn't happen in better PC/laptop.
5. Example from mediapipe website is good enough to start using the models but some details is missing from the doc. It is not a big deal because we can easily find the answers online or by reading the code from mediapipe.
